In [1]:
pip install tensorflow tensorflow-datasets matplotlib

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [5]:
# Load EMNIST Dataset (letters)
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

BATCH_SIZE = 128

ds_train = ds_train.map(normalize_img).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(normalize_img).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/letters/incomplete.KVA489_3.1.0/emnist-train.tfrecord*...:   0%|   …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/letters/incomplete.KVA489_3.1.0/emnist-test.tfrecord*...:   0%|    …

Dataset emnist downloaded and prepared to /root/tensorflow_datasets/emnist/letters/3.1.0. Subsequent calls will reuse this data.


In [3]:
rnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((28, 28), input_shape=(28, 28, 1)),
    tf.keras.layers.SimpleRNN(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(27, activation='softmax')  # 26 letters + 1 blank
])

rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 28, 28)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 128)                 │          20,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 27)                  │           3,483 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,091 (156.61 KB)

 Trainable params: 40,091 (156.61 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
rnn_history = rnn_model.fit(ds_train, epochs=10, validation_data=ds_test)

Epoch 1/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 33s 44ms/step - accuracy: 0.3871 - loss: 2.0234 - val_accuracy: 0.6992 - val_loss: 0.9227
Epoch 2/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 34s 35ms/step - accuracy: 0.7377 - loss: 0.8112 - val_accuracy: 0.7787 - val_loss: 0.6845
Epoch 3/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 42s 36ms/step - accuracy: 0.7880 - loss: 0.6580 - val_accuracy: 0.7952 - val_loss: 0.6242
Epoch 4/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.8151 - loss: 0.5759 - val_accuracy: 0.7948 - val_loss: 0.6360
Epoch 5/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 23s 33ms/step - accuracy: 0.8288 - loss: 0.5289 - val_accuracy: 0.8118 - val_loss: 0.5867
Epoch 6/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 44s 37ms/step - accuracy: 0.8383 - loss: 0.4998 - val_accuracy: 0.8066 - val_loss: 0.6020
Epoch 7/10
674/694 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8430 - loss: 0.4829

In [ ]:
cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(27, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

In [ ]:
cnn_history = cnn_model.fit(ds_train, epochs=10, validation_data=ds_test)

In [ ]:
plt.plot(rnn_history.history['val_accuracy'], label='RNN Validation Accuracy')
plt.plot(cnn_history.history['val_accuracy'], label='CNN Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.title('Character Recognition: RNN vs CNN')
plt.show()